In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers.models.layoutlmv3.modeling_layoutlmv3 import (
    LayoutLMv3ForSequenceClassification, # noqa
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/layoutlmv3-base")
layoutlmv3 = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/layoutlmv3-base", num_labels=2
)

Some weights of LayoutLMv3ForSequenceClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
text = ["This", "is", "a", "test", "sentence", "."]
bboxes = [[100, 100, 150, 150], [160, 100, 200, 150], [210, 100, 240, 150], [250, 100, 290, 150], [300, 100, 370, 150], [380, 100, 400, 150]]
inputs = tokenizer(
    text,
    boxes=bboxes,
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512,
)
with torch.no_grad():
    output = layoutlmv3(**inputs)
    print(output.logits.shape)

torch.Size([1, 2])


In [30]:


# @auto_docstring
# def forward(
#     self,
#     input_ids: Optional[torch.LongTensor] = None,
#     attention_mask: Optional[torch.FloatTensor] = None,
#     token_type_ids: Optional[torch.LongTensor] = None,
#     position_ids: Optional[torch.LongTensor] = None,
#     head_mask: Optional[torch.FloatTensor] = None,
#     inputs_embeds: Optional[torch.FloatTensor] = None,
#     labels: Optional[torch.LongTensor] = None,
#     output_attentions: Optional[bool] = None,
#     output_hidden_states: Optional[bool] = None,
#     return_dict: Optional[bool] = None,
#     bbox: Optional[torch.LongTensor] = None,
#     pixel_values: Optional[torch.LongTensor] = None,
# ) -> Union[tuple, SequenceClassifierOutput]:
act = {}
def get_hook(name):
    def hook(m, input, output):
        act[name] = output
    return hook

from torch import nn
class ModelWrapper  (nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, inputs):
        from torch.nn.functional import softmax
        outputs = self.model(inputs)
        logits = outputs.logits
        return softmax(logits, dim=-1)

model = ModelWrapper(layoutlmv3)

In [31]:
# from torchxai.explainers import SaliencyExplainer
# explainer = SaliencyExplainer(model=model)
# explainer.explain(
#     inputs=torch.randint(0, 1000, (1, 512)),
#     target=0,)

In [35]:
act = {}
def get_hook(name):
    def hook(m, input, output):
        act[name] = output
    return hook
model.model.layoutlmv3.embeddings.word_embeddings.register_forward_hook(get_hook("embeddings"))


with torch.autograd.set_grad_enabled(True):
    # runs forward pass
    outputs = model(inputs['input_ids'])
    outputs = outputs[:, 1]  # select class 1
    print('act',act['embeddings'].shape)
    assert outputs[0].numel() == 1, (
        "Target not provided when necessary, cannot"
        " take gradient with respect to multiple outputs."
    )
    # torch.unbind(forward_out) is a list of scalar tensor tuples and
    # contains batch_size * #steps elements
    grads = torch.autograd.grad(torch.unbind(outputs), act['embeddings'])

print('grads',grads)
# for name, param in model.named_parameters():
#     print("param {}, grad {}".format(name, param.grad))

/mnt/noel/phd-2026/projects/atria/.venv/lib/python3.11/site-packages/transformers/modeling_utils.py:1731: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


act torch.Size([1, 512, 768])
grads (tensor([[[ 1.0254e-03,  1.3341e-03,  3.6087e-04,  ...,  3.9355e-04,
          -1.3094e-03, -1.1965e-03],
         [-6.1949e-06,  1.3573e-04,  1.4082e-05,  ...,  2.8604e-05,
           6.1221e-05,  4.5288e-05],
         [ 3.4137e-05,  2.5224e-05, -2.0394e-05,  ...,  1.5271e-06,
           4.2496e-05,  3.4269e-05],
         ...,
         [-1.1181e-06, -1.4219e-05, -3.1183e-05,  ...,  2.2516e-05,
          -3.2169e-05, -2.8308e-05],
         [-1.1301e-06, -1.4247e-05, -3.1193e-05,  ...,  2.2515e-05,
          -3.2181e-05, -2.8297e-05],
         [-1.1397e-06, -1.4184e-05, -3.1220e-05,  ...,  2.2456e-05,
          -3.2236e-05, -2.8266e-05]]]),)


In [ ]:
from transformers.models.bert.modeling_bert import BertForSequenceClassification